# Nature Conservancy Fisheries Kaggle Competition
## Training Backdoor for manual additions to training set
This script can be used in the background to FISH_FINDER_TRAINING to observe staged fovea and commit them to the training set manually.  
#### Dependencies

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
from scipy import misc
import os
import fish_data as fd
import matplotlib.pyplot as plt
% matplotlib inline

#### Load Metadata for run

In [ ]:
version_ID = 'PILOT_RUN_v5'

In [ ]:
wd = os.getcwd()
md = wd+'/FISHFINDER_MODELS/'+version_ID
if not os.path.exists(md) :
    os.makedirs(md)


with open(md+'/meta_dictionary.pickle', 'rb') as  handle :
    meta_dict = pickle.load(handle)


### Graphical Analysis of Training Progress

In [ ]:
colors = ['red', 'darkred', 'orange', 'purple', 'blue', 'darkblue', 'green', 'darkgreen']
fish = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

epoch_completed, staged, training, keep_threshold, weights  = [], [], [], [], []
for x in meta_dict :
    if x != 0 :
        epoch_completed.append(x)
        staged.append(meta_dict.get(x).get('stage_set'))
        keep_threshold.append(meta_dict.get(x).get('keep_threshold'))
        training.append(meta_dict.get(x).get('training_set'))
        weights.append(meta_dict.get(x).get('count_weights'))

f, axarr = plt.subplots(3,2, figsize=(14,10))
axarr[0,0].set_title('Number of staged fovea')
axarr[0,0].plot(epoch_completed, staged, linestyle = '-')
axarr[1,0].plot(epoch_completed, keep_threshold, linestyle = '-')
axarr[1,0].set_title('"Keep" threshold for confidence of prediction')
axarr[0,1].set_title("Number of training set fovea")
axarr[0,1].plot(epoch_completed, training, linestyle = '-')
axarr[0,1].set_ylim(13300, 15500)
axarr[0,0].set_xlabel('Epochs completed')
axarr[1,0].set_xlabel('Epochs completed')
axarr[0,1].set_xlabel('Epochs completed')
axarr[2,0].bar(np.arange(8), np.squeeze(np.array(weights), 1)[np.argmax(epoch_completed),:])
axarr[2,0].set_xticks(np.arange(8))
axarr[2,0].set_xticklabels(fish)
axarr[2,0].set_title('Cost Weights for Class Frequency')
axarr[2,0].set_yscale('log')

for i in range(8) :
    axarr[2,1].plot(epoch_completed, np.squeeze(weights, 1)[:,i], c = colors[i], linestyle = '--', lw = 1, label = fish[i])
axarr[2,1].set_title('Change in Cost Weights per fish class over training')
axarr[2,1].set_xlabel('Epochs completed')
axarr[2,1].legend(bbox_to_anchor=(1.05, 1), loc = 2)
plt.tight_layout()
plt.show()

#### Load fovea dictionaries

In [ ]:
with open(md+'/staged_dictionary.pickle', 'rb') as handle :
    staged_dictionary = pickle.load(handle)
print(len(staged_dictionary))

In [ ]:
with open(md+'/training_dictionary.pickle', 'rb') as handle : 
    training_set_dictionary = pickle.load(handle)
print(len(training_set_dictionary))


#### Analyze fovea and commit fovea via manual inspection

In [ ]:
fd.manual_stage_manager(staged_dictionary, training_set_dictionary, 224, stage_step_threshold = 3, md = md)

In [ ]:
with open(md+'/staged_dictionary.pickle', 'wb') as fsd:
    pickle.dump(staged_dictionary, fsd)
with open(md+'/training_dictionary.pickle', 'wb') as ftd:
    pickle.dump(training_set_dictionary, ftd)